In [1]:
import pandas as pd
from pathlib import Path

In [2]:
class_list = pd.read_csv("data/raw/class_list.csv")
timeslots = pd.read_csv("data/raw/timeslots.csv")
student_classes = pd.read_csv("data/raw/student_classes.csv")
teacher_classes = pd.read_csv("data/raw/teacher_classes.csv")
lessons_required = pd.read_csv("data/raw/lessons_required.csv")

teachers = pd.read_csv("data/bronze/bronze_teachers.csv")
students = pd.read_csv("data/bronze/bronze_students.csv")

In [3]:
# Create subject_grade DataFrame from class_list
# Split the 'class' column into subject and grade
subject_grade = class_list.copy()

# Safely split even if some rows don't contain '-'
split_cols = subject_grade['class'].str.rsplit('-', n=1, expand=True)

# Assign column names
subject_grade['subject'] = split_cols[0].str.strip()
subject_grade['grade'] = split_cols[1].str.strip()

# Convert grade to integer
subject_grade['grade'] = pd.to_numeric(subject_grade['grade'], errors='coerce')

# Create unique subjects/grade DataFrame
subjects = subject_grade[['subject']].drop_duplicates().reset_index(drop=True)
grades = subject_grade[['grade']].drop_duplicates().reset_index(drop=True)

In [4]:
# Create period_interval DataFrame from timeslots
# Split the 'timeslot' column into start_time and end_time
period_interval = timeslots.copy()

# Split on ' - '
split_cols = period_interval['timeslot'].str.split(' - ', n=1, expand=True)

# Clean start_time (remove weekday, replace '_' with ':')
period_interval['start_time'] = (
    split_cols[0]
    .str.replace(r'^[A-Z]+\s*', '', regex=True)  # drop weekday like 'FRI'
    .str.replace('_', ':', regex=False)          # turn 07_45 -> 07:45
    .str.strip()
)

# Clean end_time (just replace '_' with ':')
period_interval['end_time'] = (
    split_cols[1]
    .str.replace('_', ':', regex=False)
    .str.strip()
)

# Convert both to datetime (time-only)
period_interval['start_time'] = pd.to_datetime(period_interval['start_time'], format='%H:%M', errors='coerce').dt.time
period_interval['end_time']   = pd.to_datetime(period_interval['end_time'], format='%H:%M', errors='coerce').dt.time

# Keep only start and end
period_interval = period_interval[['start_time', 'end_time']].drop_duplicates()

In [5]:
# Create subject_grade DataFrame from class_list
# Split the 'class' column into subject and grade
learners = student_classes.copy()

# Safely split even if some rows don't contain '-'
split_cols = learners['class'].str.rsplit('-', n=1, expand=True)

# Assign column names
learners['subject'] = split_cols[0].str.strip()
learners['grade'] = split_cols[1].str.strip()

# Convert grade to integer
learners['grade'] = pd.to_numeric(learners['grade'], errors='coerce')
# Join learners with students on 'student'
merged = pd.merge(students, learners, on="student", how="inner")

student_grade = merged[['student_id', 'grade']].drop_duplicates().reset_index(drop=True)
student_subject = merged[['student_id', 'subject']].drop_duplicates().reset_index(drop=True)

In [6]:
# Create subject_grade DataFrame from class_list
# Split the 'class' column into subject and grade
teacher = teacher_classes.copy()

# Safely split even if some rows don't contain '-'
split_cols = teacher['class'].str.rsplit('-', n=1, expand=True)

# Assign column names
teacher['subject'] = split_cols[0].str.strip()
teacher['grade'] = split_cols[1].str.strip()

# Convert grade to integer
teacher['grade'] = pd.to_numeric(teacher['grade'], errors='coerce')
# Join teacher with teachers on 'teacher'
merged = pd.merge(teachers, teacher, on="teacher", how="inner")

teacher_grade = merged[['teacher_id', 'grade']].drop_duplicates().reset_index(drop=True)
teacher_subject = merged[['teacher_id', 'subject']].drop_duplicates().reset_index(drop=True)

In [7]:
# Safely split even if some rows don't contain '-'
split_cols = lessons_required['class'].str.rsplit('-', n=1, expand=True)

# Assign column names
lessons_required['subject'] = split_cols[0].str.strip()
lessons_required['grade'] = split_cols[1].str.strip()

# Convert grade to integer
lessons_required['grade'] = pd.to_numeric(lessons_required['grade'], errors='coerce')

lessons_required = lessons_required[['subject', 'grade', 'num_lessons']].drop_duplicates().reset_index(drop=True)

In [8]:
# Output folder
output_path = Path("data/bronze")
output_path.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist

# Dictionary of tables
tables = {
    'bronze_subjects': subjects,
    'bronze_grades': grades,
    'bronze_period_interval': period_interval,
    'bronze_student_grade': student_grade,
    'bronze_student_subject': student_subject,
    'bronze_teacher_grade': teacher_grade,
    'bronze_teacher_subject': teacher_subject,
    'bronze_lessons_required': lessons_required
}

# Save each DataFrame to CSV
for table_name, table in tables.items():
    table.to_csv(output_path / f"{table_name}.csv", index=False)
    print(f"{table_name}.csv saved.")

print("All CSVs saved successfully.")

bronze_subjects.csv saved.
bronze_grades.csv saved.
bronze_period_interval.csv saved.
bronze_student_grade.csv saved.
bronze_student_subject.csv saved.
bronze_teacher_grade.csv saved.
bronze_teacher_subject.csv saved.
bronze_lessons_required.csv saved.
All CSVs saved successfully.
